# ⚙️ SmartBin — Optimització amb Transfer Learning i Optuna *(Versió 2.3)*

> **Aquest notebook ha estat generat amb assistència d’intel·ligència artificial, però totes les decisions sobre les tècniques i metodologies utilitzades han estat preses per una persona humana.**

---

## 🧩 Descripció general

Aquesta versió del projecte **SmartBin** implementa un sistema complet de **classificació d’imatges de residus** basat en **Transfer Learning** amb el model **ResNet18**, optimitzat automàticament mitjançant **Optuna**.  
El notebook combina tècniques avançades d’augmentació de dades, balanceig de classes i optimització d’hiperparàmetres per obtenir un model robust, eficient i capaç de generalitzar bé en diferents condicions visuals.

El sistema està dissenyat per classificar **sis categories de residus**:
- **Plàstics**
- **Metalls**
- **Paper**
- **Vidre**
- **Orgànic**
- **Tèxtil**

Aquest enfocament permet al SmartBin reconèixer una gamma més àmplia de materials, millorant així la seva capacitat de gestió selectiva i el seu potencial d’ús real.

> **Dataset utilitzat:** [Recyclable and Household Waste Classification (Kaggle)](https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification)

---

## 🎯 Objectiu del projecte

L’objectiu d’aquesta versió és **millorar significativament la precisió de classificació** respecte a les versions anteriors, aprofitant els avantatges del *transfer learning* i l’optimització automàtica d’hiperparàmetres.  
En concret, es busca reduir la pèrdua de validació i augmentar el F1-score mitjançant:

- **Transfer Learning amb ResNet18:** utilitzant pesos preentrenats sobre ImageNet per accelerar l’aprenentatge.  
- **Augmentació selectiva de dades:** per incrementar la representació de classes minoritàries (*vidre* i *tèxtil*).  
- **Pesatge de classes (WeightedRandomSampler):** per compensar desequilibris en la distribució del dataset.  
- **Entrenament amb pruning i early stopping:** que milloren l’eficiència i eviten sobreajustament.  
- **Generació automàtica d’informes:** amb mètriques, matrius de confusió i històrics d’optimització.

Aquest conjunt d’estratègies permet obtenir un model **més precís, equilibrat i escalable**, preparat per a aplicacions en entorns físics reals.

---

## 🗂️ Dataset i preparació de dades

El dataset s’organitza en **carpetes per classe**, seguint una estructura simple i funcional.  
La divisió s’ha realitzat de manera automàtica en tres subconjunts:

- **60% entrenament**  
- **20% validació**  
- **20% test**

Durant el preprocessament s’apliquen transformacions específiques:

- **Entrenament:**  
  - Rotacions aleatòries, reflexions horitzontals, canvis de color i recadrament aleatori.  
  - Augmentació extra per a les classes menys representades (*vidre* i *tèxtil*).  

- **Validació i Test:**  
  - Redimensionament a 224×224 píxels i normalització segons els valors estàndard d’ImageNet.

A més, s’empra un **WeightedRandomSampler** per assegurar que totes les classes tinguin una presència equilibrada durant l’entrenament, evitant que el model s’inclini cap a categories majoritàries.

---

## 🧠 Arquitectura del model: ResNet18 amb Transfer Learning

El model base és una **ResNet18** preentrenada amb milions d’imatges d’ImageNet.  
Mitjançant *transfer learning*, s’aprofiten les primeres capes per conservar el coneixement general de característiques visuals, adaptant només les últimes capes a la nova tasca de classificació de residus.

L’estratègia aplicada és la següent:

- **Congelació parcial de capes:**  
  Les primeres capes convolucionals (`conv1`, `bn1`, `layer1`, `layer2`) es mantenen fixes per conservar els pesos preapresos.  

- **Desbloqueig selectiu:**  
  Les capes superiors (`layer3`, `layer4` i `fc`) es reentrenen per aprendre patrons específics del nou dataset.

- **Capa de sortida modificada:**  
  S’adapta la capa final fully connected per generar **6 classes** de sortida.

- **Dropout ajustable:**  
  Per reduir l’overfitting, es permet optimitzar el valor de *dropout* durant la cerca d’hiperparàmetres.

Aquesta combinació permet **reduir el temps d’entrenament** i **millorar la precisió final** amb un ús eficient de recursos.

---

## ⚙️ Optimització automàtica amb Optuna

Per ajustar els hiperparàmetres, s’utilitza la llibreria **Optuna**, que realitza una optimització automàtica basada en tècniques d’estimació bayesiana i pruning adaptatiu.  
Cada *trial* representa un entrenament complet del model amb una combinació diferent d’hiperparàmetres, buscant **minimitzar la pèrdua de validació (validation loss)**.

| Hiperparàmetre | Rang o valors explorats |
|-----------------|--------------------------|
| `learning_rate` | 1e-5 – 1e-3 (escala logarítmica) |
| `batch_size` | [16, 24, 32] |
| `num_epochs` | 5 – 12 |
| `dropout` | 0.3 – 0.6 (increments de 0.05) |

### 🔬 Tècniques complementàries per millorar l’eficiència

- **Pruning automàtic:** interromp *trials* amb resultats no prometedors, estalviant temps computacional.  
- **Early stopping:** atura l’entrenament si no hi ha millora durant 5 èpoques consecutives.  
- **Scheduler dinàmic:** ajusta el *learning rate* automàticament amb `ReduceLROnPlateau`.  

Es duen a terme **20 trials**, que equivalen a unes 4–6 hores d’entrenament en GPU.  
El model amb la menor pèrdua de validació es desa automàticament com el **millor resultat global**.

---

## 📈 Resultats i mètriques

Després de finalitzar l’optimització, el millor model és avaluat sobre el conjunt de **test**, generant un conjunt complet de resultats i informes:

| Fitxer | Descripció |
|--------|-------------|
| **`best_resnet_model_accXX.pth`** | Pesos del model final optimitzat |
| **`resnet_model_config.json`** | Configuració dels hiperparàmetres seleccionats |
| **`confusion_matrix.png`** | Matriu de confusió per classe |
| **`optimization_report.json`** | Informe complet amb mètriques i resultats per classe |
| **`optimization_history.png`** | Evolució de la pèrdua i l’accuracy durant la optimització |
| **`class_distribution.png`** | Distribució visual del dataset |

L’informe inclou:
- Precisió (*accuracy*), *recall*, *f1-score* i *support* per classe.  
- Avaluació comparativa entre classes amb augmentació extra i classes estàndard.  
- Identificació de les categories amb millor i pitjor rendiment.

---

In [3]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, models
from PIL import Image
import os
import random
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import json
from datetime import datetime
from collections import Counter

# Dataset Class amb Augmentació Específica per Classes
class WasteDataset6Classes(Dataset):
    def __init__(self, root_dir, split, transform=None, val_transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.val_transform = val_transform
        self.split = split
        
        # 6 classes de residus
        self.classes = ['plastics', 'metalls', 'paper', 'vidre', 'organic', 'textil']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Classes que necessiten augmentació extra
        self.extra_augmentation_classes = {'vidre', 'textil'}
        
        self.image_paths = []
        self.labels = []
        
        print(f"   🔄 Creant dataset {split} amb 6 classes de residus...")
        
        # Carregar imatges per cada classe
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                print(f"   ⚠️ Advertència: No es troba la carpeta {class_dir}")
                continue
                
            # Obtenir totes les imatges de la classe
            image_names = []
            for file in os.listdir(class_dir):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_names.append(file)
            
            # Barrejar per splits consistents
            random.seed(42)
            random.shuffle(image_names)
            
            # Dividir segons split (60/20/20)
            total_images = len(image_names)
            train_end = int(0.6 * total_images)
            val_end = int(0.8 * total_images)
            
            if split == 'train':
                selected_images = image_names[:train_end]
            elif split == 'val':
                selected_images = image_names[train_end:val_end]
            else:  # test
                selected_images = image_names[val_end:]
            
            # Afegir al dataset
            for image_name in selected_images:
                full_path = os.path.join(class_dir, image_name)
                self.image_paths.append(full_path)
                self.labels.append(self.class_to_idx[class_name])
        
        # Estadístiques del dataset
        label_counts = Counter(self.labels)
        class_distribution = {self.classes[i]: label_counts[i] for i in range(len(self.classes))}
        
        print(f"   ✅ Dataset {split} creat: {len(self.image_paths)} imatges")
        print(f"   📊 Distribució per classe: {class_distribution}")
        
        # Crear transformacions d'augmentació extra per vidre i textil
        if split == 'train':
            self.extra_transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.RandomCrop((224, 224)),
                transforms.RandomHorizontalFlip(p=0.6),
                transforms.RandomVerticalFlip(p=0.3),
                transforms.RandomRotation(degrees=45),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
                transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2)),
                transforms.RandomGrayscale(p=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        class_name = self.classes[label]
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error carregant {image_path}: {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        # Aplicar transformacions segons el split i la classe
        if self.split == 'train' and self.transform:
            # Si és classe vidre o textil, aplicar augmentació extra
            if class_name in self.extra_augmentation_classes:
                image = self.extra_transform(image)
            else:
                image = self.transform(image)
        elif self.val_transform and self.split in ['val', 'test']:
            image = self.val_transform(image)
        
        return image, label
    
    def get_class_weights(self):
        """Calcula els pesos per WeightedRandomSampler"""
        label_counts = Counter(self.labels)
        total_samples = len(self.labels)
        
        # Calcular pesos inversament proporcionals a la freqüència
        class_weights = []
        for i in range(len(self.classes)):
            count = label_counts[i] if i in label_counts else 1
            weight = total_samples / (len(self.classes) * count)
            class_weights.append(weight)
        
        # Crear pesos per cada mostra
        sample_weights = [class_weights[label] for label in self.labels]
        
        return torch.DoubleTensor(sample_weights)

# Model ResNet18 amb Transfer Learning
class ResNetWasteClassifier(nn.Module):
    def __init__(self, num_classes=6, dropout=0.5):
        super(ResNetWasteClassifier, self).__init__()
        
        # Carregar ResNet18 preentrenada (fix for deprecation warning)
        try:
            # Try new weights parameter first
            self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except:
            # Fall back to pretrained parameter
            self.resnet = models.resnet18(pretrained=True)
        
        # Congelar primeres capes (conv1, layer1, layer2)
        for param in self.resnet.conv1.parameters():
            param.requires_grad = False
        for param in self.resnet.bn1.parameters():
            param.requires_grad = False
        for param in self.resnet.layer1.parameters():
            param.requires_grad = False
        for param in self.resnet.layer2.parameters():
            param.requires_grad = False
        
        # Deixar entrenables layer3, layer4
        for param in self.resnet.layer3.parameters():
            param.requires_grad = True
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
        
        # Modificar la fully connected layer
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(num_features, num_classes)
        )
        
        print(f"   🧠 ResNet18 inicialitzada amb {num_classes} classes i dropout={dropout}")
        print(f"   🔒 Congelades: conv1, bn1, layer1, layer2")
        print(f"   🔓 Entrenables: layer3, layer4, fc")
    
    def forward(self, x):
        return self.resnet(x)

def evaluate_model(model, test_loader, criterion, device, class_names):
    """Avalua el model i retorna mètriques detallades"""
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0
    
    print("🔍 Avaluant model en dataset de test...")
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Avaluació"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    return {
        'test_loss': test_loss,
        'accuracy': accuracy,
        'predictions': all_preds,
        'true_labels': all_labels,
        'classification_report': classification_report(all_labels, all_preds, 
                                                     target_names=class_names, 
                                                     output_dict=True, zero_division=0)
    }

def plot_confusion_matrix(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Genera i guarda la matriu de confusió"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriu de Confusió - ResNet18 Optimitzat', fontsize=16, pad=20)
    plt.ylabel('Etiquetes Reals', fontsize=12)
    plt.xlabel('Prediccions', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 Matriu de confusió guardada a: {save_path}")

def generate_report(study, best_metrics, class_names, save_path='optimization_report.json'):
    """Genera un informe complet de l'optimització"""
    report = {
        'optimization_info': {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'project': 'Classificació de Residus - ResNet18 + Optuna',
            'model_architecture': 'ResNet18 (Transfer Learning)',
            'frozen_layers': ['conv1', 'bn1', 'layer1', 'layer2'],
            'trainable_layers': ['layer3', 'layer4', 'fc'],
            'dataset_info': {
                'classes': class_names,
                'total_classes': len(class_names),
                'data_split': '60/20/20 (train/val/test)',
                'extra_augmentation': ['vidre', 'textil']
            },
            'n_trials': len(study.trials),
            'best_trial_number': study.best_trial.number,
            'optimization_direction': 'minimize_validation_loss'
        },
        'best_hyperparameters': study.best_trial.params,
        'best_validation_loss': study.best_trial.value,
        'test_metrics': {
            'test_loss': best_metrics['test_loss'],
            'test_accuracy': best_metrics['accuracy']
        },
        'detailed_classification_report': best_metrics['classification_report'],
        'trial_history': [
            {
                'trial': t.number,
                'params': t.params,
                'value': t.value,
                'state': str(t.state)
            } for t in study.trials if t.value is not None
        ]
    }
    
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"📄 Informe complet guardat a: {save_path}")
    return report

def plot_optimization_history(study, save_path='optimization_history.png'):
    """Genera gràfic de la història d'optimització"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Història de valors
    trial_numbers = [t.number for t in study.trials if t.value is not None]
    values = [t.value for t in study.trials if t.value is not None]
    
    ax1.plot(trial_numbers, values, 'b-o', markersize=6, linewidth=2)
    ax1.axhline(y=study.best_value, color='r', linestyle='--', linewidth=2, 
                label=f'Millor: {study.best_value:.4f}')
    ax1.set_xlabel('Trial Number')
    ax1.set_ylabel('Validation Loss')
    ax1.set_title('Història d\'Optimització ResNet18')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribució de valors
    if len(values) > 0:
        ax2.hist(values, bins=min(15, len(values)), alpha=0.7, color='lightblue', 
                edgecolor='darkblue', linewidth=1)
        ax2.axvline(x=study.best_value, color='r', linestyle='--', linewidth=2, 
                   label=f'Millor: {study.best_value:.4f}')
    ax2.set_xlabel('Validation Loss')
    ax2.set_ylabel('Freqüència')
    ax2.set_title('Distribució de Resultats')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📈 Gràfic d'optimització guardat a: {save_path}")

def plot_class_distribution(train_dataset, save_path='class_distribution.png'):
    """Genera gràfic de distribució de classes"""
    label_counts = Counter(train_dataset.labels)
    class_names = train_dataset.classes
    counts = [label_counts[i] for i in range(len(class_names))]
    
    # Colors personalitzats per cada classe
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']
    
    plt.figure(figsize=(12, 8))
    bars = plt.bar(class_names, counts, color=colors[:len(class_names)])
    
    # Afegir valors sobre les barres
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + max(counts)*0.01, 
                str(count), ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    plt.title('Distribució de Classes en Dataset d\'Entrenament (6 Classes)', fontsize=16, pad=20)
    plt.xlabel('Classes de Residus', fontsize=12)
    plt.ylabel('Nombre d\'Imatges', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    
    # Remove special character issue with standard text
    plt.figtext(0.02, 0.02, 
                "Extra augmentation applied to: vidre, textil", 
                fontsize=10, style='italic', color='darkred')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 Gràfic de distribució guardat a: {save_path}")

# Configuració Principal
print("🚀 INICIANDO RESNET18 + OPTUNA - CLASSIFICACIÓ 6 CLASSES DE RESIDUS")
print("="*80)

# Transformacions d'augmentació per entrenament
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformacions sense augmentació per validació i test
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Inicialitzar datasets
print("📂 Carregant datasets amb 6 classes de residus...")
dataset_path = 'images_6classes'

train_dataset = WasteDataset6Classes(dataset_path, 'train', 
                                    transform=train_transform, 
                                    val_transform=val_test_transform)
val_dataset = WasteDataset6Classes(dataset_path, 'val', 
                                  val_transform=val_test_transform)
test_dataset = WasteDataset6Classes(dataset_path, 'test', 
                                   val_transform=val_test_transform)

print(f"📊 Dataset: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)} | {len(train_dataset.classes)} classes")
print(f"🏷️ Classes: {train_dataset.classes}")
print(f"⭐ Augmentació extra: {train_dataset.extra_augmentation_classes}")

# Generar gràfic de distribució
plot_class_distribution(train_dataset)

# Calcular pesos per WeightedRandomSampler
print("⚖️ Calculant pesos per equilibrar classes...")
sample_weights = train_dataset.get_class_weights()
print(f"   📈 Pesos calculats per {len(sample_weights)} mostres d'entrenament")

# Variables globals per tracking
best_global_loss = float('inf')
best_model_state = None
trial_count = 0

def objective(trial):
    global best_global_loss, best_model_state, trial_count
    trial_count += 1
    
    # Hiperparàmetres a optimitzar
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 24, 32])
    epochs = trial.suggest_int('num_epochs', 5, 12)
    dropout = trial.suggest_float('dropout', 0.3, 0.6, step=0.05)
    
    print(f"\n🔍 Trial {trial_count}/20")
    print(f"   📊 Paràmetres: lr={lr:.6f}, bs={batch_size}, epochs={epochs}, dropout={dropout:.2f}")
    
    # Crear DataLoaders amb WeightedRandomSampler
    weighted_sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        sampler=weighted_sampler,
        num_workers=0, 
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=0, 
        pin_memory=False
    )
    
    # Model setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNetWasteClassifier(num_classes=len(train_dataset.classes), dropout=dropout).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Learning rate scheduler - FIXED: removed verbose parameter
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 5
    
    print(f"   🚀 Iniciant entrenament ResNet18...")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"   Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Actualitzar barra de progrés
            current_train_loss = train_loss / (train_bar.n + 1)
            current_train_acc = 100. * train_correct / train_total
            train_bar.set_postfix({'Loss': f'{current_train_loss:.4f}', 'Acc': f'{current_train_acc:.1f}%'})
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100. * train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"   Epoch {epoch+1}/{epochs} [Val]  ", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Actualitzar barra de progrés
                current_val_loss = val_loss / (val_bar.n + 1)
                current_val_acc = 100. * val_correct / val_total
                val_bar.set_postfix({'Loss': f'{current_val_loss:.4f}', 'Acc': f'{current_val_acc:.1f}%'})
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        
        # Update learning rate scheduler
        scheduler.step(avg_val_loss)
        
        # Mostrar resultats de l'epoch
        improvement_indicator = ""
        if avg_val_loss < best_val_loss:
            improvement_indicator = " ⬆️"
        
        print(f"   Epoch {epoch+1}: Train={avg_train_loss:.4f}({train_accuracy:.1f}%) | Val={avg_val_loss:.4f}({val_accuracy:.1f}%){improvement_indicator}")
        
        # Report intermediate result per pruning
        trial.report(avg_val_loss, epoch)
        
        # Prune si no és prometedor
        if trial.should_prune():
            print(f"   ✂️ Trial podat - no és prometedor")
            raise optuna.TrialPruned()
        
        # Update best validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            
            # Si és el millor model global, guardar-lo
            if avg_val_loss < best_global_loss:
                best_global_loss = avg_val_loss
                best_model_state = model.state_dict().copy()
                torch.save(best_model_state, 'best_resnet_temp.pth')
                print(f"   🏆 ¡NOU RÈCORD GLOBAL! Val Loss: {avg_val_loss:.6f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"   ⏰ Early stopping (sense millora per {patience} epochs)")
            break
    
    print(f"   ✅ COMPLETAT - Millor Val Loss: {best_val_loss:.6f}")
    
    # Cleanup
    del model, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_val_loss

# Main execution with proper error handling
if __name__ == "__main__":
    # Crear estudi d'Optuna
    print(f"🎯 Objectiu: Optimitzar ResNet18 per classificació de 6 classes de residus")
    print(f"💻 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    print(f"🔒 Transfer Learning: Congelades primeres capes de ResNet18")
    print(f"⚖️ Estratègia: WeightedRandomSampler per equilibrar classes")
    print(f"⭐ Augmentació extra: vidre, textil")
    print(f"⏱️ Temps estimat: 4-6 hores per 20 trials")

    study = optuna.create_study(
        direction='minimize',
        study_name='resnet18_waste_classification',
        sampler=optuna.samplers.TPESampler(n_startup_trials=5),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
    )

    # Executar optimització
    print("🚀 Iniciant optimització...")
    try:
        study.optimize(objective, n_trials=20, timeout=21600)  # 20 trials, 6h max
        optimization_completed = True
    except KeyboardInterrupt:
        print("⏸️ Interromput per usuari")
        optimization_completed = True
    except Exception as e:
        print(f"❌ Error durant optimització: {e}")
        optimization_completed = False

    # Resultats finals - FIXED: proper error handling
    print("\n" + "="*80)
    print("🎉 OPTIMITZACIÓ RESNET18 COMPLETADA - 6 CLASSES DE RESIDUS")
    print("="*80)

    # Check if we have any successful trials
    completed_trials = [t for t in study.trials if t.value is not None]
    
    if len(completed_trials) > 0:
        best_trial = min(completed_trials, key=lambda t: t.value)
        print(f"🏆 MILLOR RESULTAT:")
        print(f"   📊 Validation Loss: {best_trial.value:.6f}")
        print(f"   ⚙️ Paràmetres: {best_trial.params}")
        print(f"   🔢 Trial número: {best_trial.number}")
        
        print(f"\n📈 Resum de trials:")
        print(f"   ✅ Trials completats: {len(completed_trials)}")
        print(f"   ✂️ Trials podats: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
        print(f"   ❌ Trials fallits: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
        
        # Avaluar el millor model en test set
        if os.path.exists('best_resnet_temp.pth'):
            print(f"\n🧪 AVALUACIÓ EN TEST SET:")
            print("-" * 60)
            
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            best_model = ResNetWasteClassifier(
                num_classes=len(train_dataset.classes), 
                dropout=best_trial.params.get('dropout', 0.5)
            ).to(device)
            best_model.load_state_dict(torch.load('best_resnet_temp.pth'))
            
            # Crear test loader
            test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
            criterion = nn.CrossEntropyLoss()
            
            test_metrics = evaluate_model(best_model, test_loader, criterion, device, train_dataset.classes)
            
            print(f"📊 Test Loss: {test_metrics['test_loss']:.6f}")
            print(f"🎯 Test Accuracy: {test_metrics['accuracy']*100:.2f}%")
            
            # Generate reports and visualizations
            plot_confusion_matrix(test_metrics['true_labels'], test_metrics['predictions'], 
                                train_dataset.classes, 'confusion_matrix.png')
            
            # Create a mock study object for reporting if needed
            mock_study_data = type('MockStudy', (), {
                'trials': study.trials,
                'best_trial': best_trial,
                'best_value': best_trial.value
            })()
            
            generate_report(mock_study_data, test_metrics, train_dataset.classes, 'optimization_report.json')
            plot_optimization_history(mock_study_data, 'optimization_history.png')
            
            # Save final model with descriptive name
            model_filename = f"best_resnet_model_acc{test_metrics['accuracy']*100:.1f}.pth"
            torch.save(best_model.state_dict(), model_filename)
            
            # Save complete model configuration
            model_config = {
                'model_info': {
                    'architecture': 'ResNet18',
                    'pretrained': True,
                    'transfer_learning': True,
                    'frozen_layers': ['conv1', 'bn1', 'layer1', 'layer2'],
                    'trainable_layers': ['layer3', 'layer4', 'fc']
                },
                'dataset_info': {
                    'num_classes': len(train_dataset.classes),
                    'classes': train_dataset.classes,
                    'extra_augmentation_classes': list(train_dataset.extra_augmentation_classes),
                    'data_split': {'train': 0.6, 'val': 0.2, 'test': 0.2}
                },
                'best_hyperparameters': best_trial.params,
                'performance_metrics': {
                    'test_accuracy': test_metrics['accuracy'],
                    'test_loss': test_metrics['test_loss'],
                    'validation_loss_best': best_trial.value
                },
                'training_info': {
                    'optimizer': 'Adam',
                    'loss_function': 'CrossEntropyLoss',
                    'scheduler': 'ReduceLROnPlateau',
                    'weighted_sampling': True,
                    'early_stopping': True
                }
            }
            
            with open('resnet_model_config.json', 'w', encoding='utf-8') as f:
                json.dump(model_config, f, indent=2, ensure_ascii=False)
            
            print("Files generated:")
            print(f"   {model_filename} - Optimized ResNet18 model")
            print("   resnet_model_config.json - Complete model configuration")
            print("   confusion_matrix.png - Confusion matrix")
            print("   optimization_report.json - Detailed optimization report")
            print("   optimization_history.png - Optimization history")
            print("   class_distribution.png - Class distribution")
            
            # Performance analysis
            print(f"\nPERFORMANCE ANALYSIS:")
            print("-" * 60)
            
            class_report = test_metrics['classification_report']
            print(f"{'Class':<10} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Support':>10}")
            print("-" * 60)
            for class_name in train_dataset.classes:
                if class_name in class_report:
                    p = class_report[class_name]['precision']
                    r = class_report[class_name]['recall']
                    f1 = class_report[class_name]['f1-score']
                    support = class_report[class_name]['support']
                    
                    marker = " *" if class_name in train_dataset.extra_augmentation_classes else ""
                    class_display = class_name + marker
                    print(f"{class_display:<10} {p:>10.3f} {r:>10.3f} {f1:>10.3f} {support:>10.0f}")
            
            # Global metrics
            if 'macro avg' in class_report:
                macro = class_report['macro avg']
                print("-" * 60)
                print(f"{'Macro Avg':<10} {macro['precision']:>10.3f} {macro['recall']:>10.3f} {macro['f1-score']:>10.3f}")
            
            if 'weighted avg' in class_report:
                weighted = class_report['weighted avg']
                print(f"{'Weighted Avg':<10} {weighted['precision']:>10.3f} {weighted['recall']:>10.3f} {weighted['f1-score']:>10.3f}")
            
            # Identify best and worst performing classes
            class_f1_scores = {}
            for class_name in train_dataset.classes:
                if class_name in class_report:
                    class_f1_scores[class_name] = class_report[class_name]['f1-score']
            
            if class_f1_scores:
                best_class = max(class_f1_scores.keys(), key=lambda x: class_f1_scores[x])
                worst_class = min(class_f1_scores.keys(), key=lambda x: class_f1_scores[x])
                
                print(f"\nBest performing class: {best_class} (F1: {class_f1_scores[best_class]:.3f})")
                print(f"Needs improvement: {worst_class} (F1: {class_f1_scores[worst_class]:.3f})")
                
                # Analyze effect of extra augmentation
                extra_classes_performance = {}
                normal_classes_performance = {}
                
                for class_name in train_dataset.classes:
                    if class_name in class_report:
                        f1_score = class_report[class_name]['f1-score']
                        if class_name in train_dataset.extra_augmentation_classes:
                            extra_classes_performance[class_name] = f1_score
                        else:
                            normal_classes_performance[class_name] = f1_score
                
                if extra_classes_performance and normal_classes_performance:
                    avg_extra = np.mean(list(extra_classes_performance.values()))
                    avg_normal = np.mean(list(normal_classes_performance.values()))
                    
                    print(f"\nEFFECT OF EXTRA AUGMENTATION:")
                    print(f"   Classes with extra augmentation (vidre, textil): F1 avg = {avg_extra:.3f}")
                    print(f"   Normal classes: F1 avg = {avg_normal:.3f}")
                    
                    if avg_extra > avg_normal:
                        print(f"   Extra augmentation improved performance (+{avg_extra-avg_normal:.3f})")
                    else:
                        print(f"   Extra augmentation did not significantly improve performance")
            
            # Instructions for using the trained model
            print(f"\nHOW TO USE THE TRAINED MODEL:")
            print("-" * 60)
            print("```python")
            print("import torch")
            print("from torchvision import transforms")
            print("import json")
            print()
            print("# Load configuration")
            print("with open('resnet_model_config.json', 'r') as f:")
            print("    config = json.load(f)")
            print()
            print("# Create model")
            print("model = ResNetWasteClassifier(")
            print(f"    num_classes={len(train_dataset.classes)},")
            print(f"    dropout={best_trial.params.get('dropout', 0.5)}")
            print(")")
            print()
            print("# Load weights")
            print(f"model.load_state_dict(torch.load('{model_filename}'))")
            print("model.eval()")
            print()
            print("# Preprocess image")
            print("transform = transforms.Compose([")
            print("    transforms.Resize((224, 224)),")
            print("    transforms.ToTensor(),")
            print("    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])")
            print("])")
            print("```")
            
            # Clean up temporary file
            if os.path.exists('best_resnet_temp.pth'):
                os.remove('best_resnet_temp.pth')
            
            print(f"\nPROCESS COMPLETED SUCCESSFULLY!")
            
        else:
            print("Warning: Temporary model file not found")
            
    else:
        print("No successful trials completed")
        print("Please verify:")
        print("   - Dataset path: 'images_6classes/'")
        print("   - Folder structure: images_6classes/[class]/[images]")
        print("   - Classes: plastics, metalls, paper, vidre, organic, textil")
        print("   - Available GPU memory")
        print("   - Image formats (PNG/JPG)")

    # Final system information
    print("\n" + "="*80)
    print("WASTE CLASSIFICATION SYSTEM INFORMATION")
    print("="*80)

    print("Waste classes (6):")
    for i, class_name in enumerate(['plastics', 'metalls', 'paper', 'vidre', 'organic', 'textil']):
        marker = " *" if class_name in ['vidre', 'textil'] else ""
        print(f"   {i}: {class_name}{marker}")

    print("\nModel architecture:")
    print("   Base: ResNet18 (pretrained on ImageNet)")
    print("   Frozen: conv1, bn1, layer1, layer2")
    print("   Trainable: layer3, layer4, fc")
    print("   Output classes: 6")

    print("\nTraining configuration:")
    print("   Dataset structure: Flat by classes")
    print("   Balancing: WeightedRandomSampler")
    print("   Augmentation: Dynamic + extra for vidre/textil")
    print("   Split: 60% train / 20% val / 20% test")
    print("   Optimization: Optuna TPE + MedianPruner")
    print("   Scheduler: ReduceLROnPlateau")
    print("   Early Stopping: Patience 5 epochs")

    print(f"\nEND OF RESNET18 OPTIMIZATION PROCESS")
    print("Model ready to classify 6 types of waste!")

    # Final cleanup
    torch.cuda.empty_cache()
    gc.collect()

🚀 INICIANDO RESNET18 + OPTUNA - CLASSIFICACIÓ 6 CLASSES DE RESIDUS
📂 Carregant datasets amb 6 classes de residus...
   🔄 Creant dataset train amb 6 classes de residus...
   ✅ Dataset train creat: 9000 imatges
   📊 Distribució per classe: {'plastics': 3300, 'metalls': 1200, 'paper': 1800, 'vidre': 900, 'organic': 1200, 'textil': 600}
   🔄 Creant dataset val amb 6 classes de residus...
   ✅ Dataset val creat: 3000 imatges
   📊 Distribució per classe: {'plastics': 1100, 'metalls': 400, 'paper': 600, 'vidre': 300, 'organic': 400, 'textil': 200}
   🔄 Creant dataset test amb 6 classes de residus...
   ✅ Dataset test creat: 3000 imatges
   📊 Distribució per classe: {'plastics': 1100, 'metalls': 400, 'paper': 600, 'vidre': 300, 'organic': 400, 'textil': 200}
📊 Dataset: Train=9000, Val=3000, Test=3000 | 6 classes
🏷️ Classes: ['plastics', 'metalls', 'paper', 'vidre', 'organic', 'textil']
⭐ Augmentació extra: {'textil', 'vidre'}


[I 2025-09-01 15:54:38,123] A new study created in memory with name: resnet18_waste_classification


📊 Gràfic de distribució guardat a: class_distribution.png
⚖️ Calculant pesos per equilibrar classes...
   📈 Pesos calculats per 9000 mostres d'entrenament
🎯 Objectiu: Optimitzar ResNet18 per classificació de 6 classes de residus
💻 Device: CUDA
🔒 Transfer Learning: Congelades primeres capes de ResNet18
⚖️ Estratègia: WeightedRandomSampler per equilibrar classes
⭐ Augmentació extra: vidre, textil
⏱️ Temps estimat: 4-6 hores per 20 trials
🚀 Iniciant optimització...

🔍 Trial 1/20
   📊 Paràmetres: lr=0.000048, bs=16, epochs=12, dropout=0.30
   🧠 ResNet18 inicialitzada amb 6 classes i dropout=0.3
   🔒 Congelades: conv1, bn1, layer1, layer2
   🔓 Entrenables: layer3, layer4, fc
   🚀 Iniciant entrenament ResNet18...


   Epoch 1: Train=0.7284(74.7%) | Val=0.6230(77.2%) ⬆️
   🏆 ¡NOU RÈCORD GLOBAL! Val Loss: 0.622989


   Epoch 2: Train=0.4312(85.1%) | Val=0.4879(84.0%) ⬆️
   🏆 ¡NOU RÈCORD GLOBAL! Val Loss: 0.487871


...

   Epoch 3: Train=0.3116(89.2%) | Val=0.4767(84.9%) ⬆️


[I 2025-09-01 18:03:32,474] Trial 19 pruned.                                                                           


   Epoch 4: Train=0.2717(90.9%) | Val=0.4936(84.8%)
   ✂️ Trial podat - no és prometedor

🎉 OPTIMITZACIÓ RESNET18 COMPLETADA - 6 CLASSES DE RESIDUS
🏆 MILLOR RESULTAT:
   📊 Validation Loss: 0.358141
   ⚙️ Paràmetres: {'learning_rate': 4.765165349547268e-05, 'batch_size': 16, 'num_epochs': 12, 'dropout': 0.3}
   🔢 Trial número: 0

📈 Resum de trials:
   ✅ Trials completats: 20
   ✂️ Trials podats: 16
   ❌ Trials fallits: 0

🧪 AVALUACIÓ EN TEST SET:
------------------------------------------------------------
   🧠 ResNet18 inicialitzada amb 6 classes i dropout=0.3
   🔒 Congelades: conv1, bn1, layer1, layer2
   🔓 Entrenables: layer3, layer4, fc
🔍 Avaluant model en dataset de test...


Avaluació: 100%|███████████████████████████████████████████████████████████████████████| 94/94 [00:15<00:00,  5.88it/s]


📊 Test Loss: 0.315282
🎯 Test Accuracy: 90.13%
📊 Matriu de confusió guardada a: confusion_matrix.png
📄 Informe complet guardat a: optimization_report.json
📈 Gràfic d'optimització guardat a: optimization_history.png
Files generated:
   best_resnet_model_acc90.1.pth - Optimized ResNet18 model
   resnet_model_config.json - Complete model configuration
   confusion_matrix.png - Confusion matrix
   optimization_report.json - Detailed optimization report
   optimization_history.png - Optimization history
   class_distribution.png - Class distribution

PERFORMANCE ANALYSIS:
------------------------------------------------------------
Class       Precision     Recall   F1-Score    Support
------------------------------------------------------------
plastics        0.930      0.905      0.918       1100
metalls         0.835      0.948      0.888        400
paper           0.881      0.913      0.897        600
vidre *         0.901      0.823      0.861        300
organic         0.949      0.8